<img src="https://www.bestdesigns.co/uploads/inspiration_images/4350/990__1511457498_404_walmart.png" alt="WALMART LOGO" />

# Walmart : predict weekly sales

## Company's Description 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## Goals 🎯

The project can be divided into three steps:

- Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2 : train a **linear regression model** (baseline)
- Part 3 : avoid overfitting by training a **regularized regression model**

## Scope of this project 🖼️

For this project, you'll work with a dataset that contains information about weekly sales achieved by different Walmart stores, and other variables such as the unemployment rate or the fuel price, that might be useful for predicting the amount of sales. The dataset has been taken from a Kaggle competition, but we made some changes compared to the original data. Please make sure that you're using **our** custom dataset (available on JULIE). 🤓

## Deliverable 📬

To complete this project, your team should: 

- Create some visualizations
- Train at least one **linear regression model** on the dataset, that predicts the amount of weekly sales as a function of the other variables
- Assess the performances of the model by using a metric that is relevant for regression problems
- Interpret the coefficients of the model to identify what features are important for the prediction
- Train at least one model with **regularization (Lasso or Ridge)** to reduce overfitting


In [481]:
## Import librairies 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

In [482]:
## Read data 
data = pd.read_csv("Walmart_Store_sales.csv")
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


## Part 1 : EDA and data preprocessing

### EDA

In [483]:
import plotly.express as px
fig= px.line(data, x="Date", y="Weekly_Sales",color="Store")
fig.show()

While doing a group by Store and Date, we can get a better visualisation of the weekly Sales

In [484]:
fig= px.line(data, x=data['Date'].unique(), y=data.groupby(["Date","Store"]).sum()['Weekly_Sales'][:86])
fig.show()

## Preprocessing to be planned with pandas

Let's see if there is any Nan values ! this is not Cheese Nan hah :D

In [485]:
data.isnull().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

We notice that we have many Nan values and we have to deal with it while preprocessing.
Our target value in this project will be Weekly_Sales. That means that we will just drop the lines in the dataset for which the value in Weekly_Sales is missing.


In [486]:
data = data[data['Weekly_Sales'].notna()]
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896


Second step will focus on converting Date type into Datetime 

In [487]:
data['date']=pd.to_datetime(data['Date'])
data=data.sort_values('date',ascending=False).drop(['Date'],axis=1)
data.head()

/var/folders/69/2xnjng8d15j5264htcrb3x640000gn/T/ipykernel_33866/3985895450.py:1: UserWarning:

Parsing '18-02-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/69/2xnjng8d15j5264htcrb3x640000gn/T/ipykernel_33866/3985895450.py:1: UserWarning:

Parsing '25-03-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/69/2xnjng8d15j5264htcrb3x640000gn/T/ipykernel_33866/3985895450.py:1: UserWarning:

Parsing '28-05-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/69/2xnjng8d15j5264htcrb3x640000gn/T/ipykernel_33866/3985895450.py:1: UserWarning:

Parsing '19-08-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/69/2xnjng8d15j5264htcrb3x640000gn/T/ipykernel_33866/3985895450.py:1: UserWarning:

Parsing '15-10-2010' in DD/MM/YYYY f

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,date
25,17.0,919878.34,0.0,44.82,3.797,131.108333,5.527,2012-12-10
22,18.0,1074079.00,0.0,50.97,4.029,138.911700,8.243,2012-12-10
87,3.0,431985.36,0.0,75.19,3.688,225.235150,6.664,2012-11-05
144,3.0,424513.08,0.0,73.44,3.594,226.968844,6.034,2012-10-19
74,1.0,1508068.77,0.0,67.97,3.594,223.425723,NaN,2012-10-19


Great, you are still here, In this part, We need to extract 'Date' into some others features like day, month, year or day of the week for preprocessing part. 

In [488]:
data['Year']=data['date'].apply(lambda x : x.year)
data['Month']=data['date'].apply(lambda x : x.month)
data['Day']=data['date'].apply(lambda d : d.day)
data['DayOfWeek']=data['date'].apply(lambda x : x.day_of_week)

data.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,date,Year,Month,Day,DayOfWeek
25,17.0,919878.34,0.0,44.82,3.797,131.108333,5.527,2012-12-10,2012.0,12.0,10.0,0.0
22,18.0,1074079.00,0.0,50.97,4.029,138.911700,8.243,2012-12-10,2012.0,12.0,10.0,0.0
87,3.0,431985.36,0.0,75.19,3.688,225.235150,6.664,2012-11-05,2012.0,11.0,5.0,0.0
144,3.0,424513.08,0.0,73.44,3.594,226.968844,6.034,2012-10-19,2012.0,10.0,19.0,4.0
74,1.0,1508068.77,0.0,67.97,3.594,223.425723,NaN,2012-10-19,2012.0,10.0,19.0,4.0


Describe our data to previsualise if we have outliers or not

In [489]:
data.describe(include="all")

/var/folders/69/2xnjng8d15j5264htcrb3x640000gn/T/ipykernel_33866/3772400700.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,date,Year,Month,Day,DayOfWeek
count,136.000000,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000,118,118.000000,118.000000,118.000000,118.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-08 00:00:00,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-10 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-10 00:00:00,NaN,NaN,NaN,NaN
mean,10.014706,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582,NaN,2010.822034,6.830508,15.949153,3.771186
std,6.124614,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428,NaN,0.812628,3.395082,8.687202,1.283765
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,NaN,2010.000000,1.000000,1.000000,0.000000
25%,4.000000,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000,NaN,2010.000000,4.000000,8.000000,4.000000


From our describe we can notice that Unemployement seems to have outliers, let's visualise that feature to be sure!

In [490]:
fig = px.box(data, y="Unemployment")
fig.show()

Let's remove outliers from that feature " Unemployment", we will remove only upper 

In [491]:
q_hi  = data["Unemployment"].quantile(0.95)

data_filtered = data[(data["Unemployment"] < q_hi)]


In [492]:
fig = px.box(data_filtered, y="Unemployment")
fig.show()

Let's begin our preprocessing with scikitlearn !

In [493]:
data=data_filtered


In [494]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = data.drop(target_variable, axis = 1)
Y = data.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
25      919878.34
22     1074079.00
87      431985.36
144     424513.08
63      313358.15
Name: Weekly_Sales, dtype: float64

X :
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
25    17.0           0.0        44.82       3.797  131.108333         5.527   
22    18.0           0.0        50.97       4.029  138.911700         8.243   
87     3.0           0.0        75.19       3.688  225.235150         6.664   
144    3.0           0.0        73.44       3.594  226.968844         6.034   
63     5.0           0.0        69.17       3.594  224.019287         5.422   

          date    Year  Month   Day  DayOfWeek  
25  2012-12-10  2012.0   12.0  10.0        0.0  
22  2012-12-10  2012.0   12.0  10.0        0.0  
87  2012-11-05  2012.0   11.0   5.0        0.0  
144 2012-10-19  2012.0   10.0  19.0        4.0  
63  2012-10-19  2012.0   10.0  19.0        4.0  


In [495]:
# Automatically detect names of numeric/categorical columns
numeric_features = [  'Day', 'Temperature', 'Fuel_Price','CPI', 'Unemployment','Year', 'Month','DayOfWeek']   # 
categorical_features = ['Store', 'Holiday_Flag']

print('Our numeric features ', numeric_features)
print('Our categorical features ', categorical_features)

Our numeric features  ['Day', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'DayOfWeek']
Our categorical features  ['Store', 'Holiday_Flag']


## Part 2 : Baseline model (linear regression)

### Preprocessing Train Test Split

In [496]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [497]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [498]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [499]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [500]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
145   14.0           0.0        72.62       2.780  182.442420         8.899   
103    4.0           0.0        42.40       2.860  126.793400         7.127   
118    9.0           NaN        82.99       2.637  215.016648         6.384   
89     2.0           0.0        76.42       3.786  215.154482         7.931   
14    17.0           0.0        60.07       2.853  126.234600         6.885   

          date    Year  Month   Day  DayOfWeek  
145 2010-06-18  2010.0    6.0  18.0        4.0  
103 2010-10-12  2010.0   10.0  12.0        1.0  
118 2010-06-18  2010.0    6.0  18.0        4.0  
89         NaT     NaN    NaN   NaN        NaN  
14  2010-01-10  2010.0    1.0  10.0        6.0  
...Done.
[[ 2.67532322e-01  7.34549836e-01 -1.25491651e+00  9.02122799e-02
   1.54151991e+00 -1.17615783e+00 -2.50820849e-01  1.62586434e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+0

### Train model

In [501]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [502]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[2119412.27946854 2233543.52539697  464537.31154339 1741650.8309885
  853340.78334255  564131.41035176  908424.26697459  409892.65927074
  327778.28240831  470976.53672464 2045441.20869774  605151.52041924
 1500025.8099704  1746388.00680112 1958000.11790363 1489631.96658691
 2052507.82466469  504162.78334744  312879.93932443  327001.75653149
  425103.05823109 1932787.65473879  669227.04822331 1512000.57341309
 1931141.94906242  624171.62168638  485471.25008253 1119568.27490169
 1976170.38880261  183958.79600096 2197764.1560563   341384.79302624
  484967.11936983 1335870.20246227  508959.24398229  785508.86636357
  445509.77265641 2098109.738085   1487985.2022932   337192.46074185
 1449813.67672302  966455.90025642  335253.65911917 2061912.99528827
  997083.19652343 1515367.51994334 1786647.27168573 2051987.79262881
  534007.68365924 1997941.88053907 1437449.1215013   615191.97322047
 1615765.26485063 1476418.63937006  990175.33227125  452159.5798

In [503]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[1309002.23465046  938398.01977777  997435.56305522  219524.67696064
  504835.36737883 1625496.04415535 1536112.29500807 1678421.94662503
 1579525.21930197 1176361.58707496 1819079.84671379  465897.51330104
  507109.17137266 1943708.79276188 2204401.44836536  668949.66245382
 2024587.93932075 1895896.77994861 1526959.98457528 1402081.64455556
 1276319.29059326  475728.03328931 2061767.60056025]



In [504]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9756258740439652
R2 score on test set :  0.8992103187694559


In [505]:
regressor.coef_

array([  -22889.89165691,   -13433.09889997,   -33835.53133989,
          53900.2220824 ,  -122183.51014868,   -41020.31649674,
          12483.81885997,   -12887.01808722,   188121.48945391,
       -1292855.45405184,   504634.8824101 , -1498181.91709804,
        -152987.46994517,  -955668.36883122,  -960898.1623095 ,
       -1358464.474165  ,   434584.25888612,  -163593.03454249,
         429130.79714475,   616102.74025585,  -771866.63926637,
       -1230654.0892163 ,  -808559.10271659,  -304652.36022874,
         -42172.81708255,   270589.16859637,   -43715.30683048,
         -42500.18526496])

In [506]:
# # Create a pandas DataFrame
# coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
# coefs

In [507]:
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Month,1.248382e+04
DayOfWeek,1.288702e+04
Temperature,1.343310e+04
Day,2.288989e+04
Fuel_Price,3.383553e+04
Year,4.102032e+04
Store_19.0,4.217282e+04
Holiday_Flag_nan,4.250019e+04
Holiday_Flag_1.0,4.371531e+04
CPI,5.390022e+04


In [508]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

## Part 3 : Fight overfitting

Cross-validated score for a Ridge model (with default value of λ

In [509]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("3-fold cross-validation...")
regressor = Ridge()
scores = cross_val_score(regressor, X_train, Y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.7887256003983847
The standard deviation is :  0.04085794627550975


Grid search : tune λ


In [510]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.8495798852385775


In [511]:
# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

R2 score on training set :  0.9730121124194345
R2 score on test set :  0.9091128411708459
